In [82]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random

# from scipy.fftpack import fft, ifft
# from scipy import signal
from sklearn import preprocessing
from sklearn.model_selection import KFold
# from sklearn.model_selection import train_test_split
# from sklearn.model_selection import KFold

# import features
pd.set_option('precision', 30)
np.set_printoptions(precision = 30)


In [7]:
train_df = pd.read_csv('../../train/train.csv', dtype={'acoustic_data': np.int8, 'time_to_failure': np.float64})

KeyboardInterrupt: 

In [ ]:
float_data = train_df.values

In [83]:
# del train_df
train_df = pd.read_csv('diff_format.csv')


In [84]:
# train_df.head()
# del train_df

In [85]:
float_data=train_df.values[:,:-1]
y = train_df.values[:,-1]
del train_df


In [86]:
# print(float_data[0])
# print(y[0])
print(y.shape)
float_data.shape

(629145,)


(629145, 6)

In [87]:
scaler = preprocessing.StandardScaler()
scaler.fit(float_data)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [88]:
# print(scaler.transform(float_data))

In [89]:
scaled_X = scaler.transform(float_data)

In [90]:
kf = KFold(n_splits=5)

In [91]:
kf

KFold(n_splits=5, random_state=None, shuffle=False)

In [92]:
def x_y_split(data):
    x = data[:,0]
    y = data[:,1]
    return x, y

def validation_split(data, ratio):
    slice_index = int(len(data) * ratio)
    train_data = data[:slice_index]
    valid_set = data[slice_index:]
    return train_data, valid_set

def create_segments(data):
    size = 150000
    segment = []
    for i in range(0, len(data), size):
        segment.append(data[i:i+size])
    return segment


In [93]:
class KerasDataGenerator(object):

    def __init__(self, X, y, n_steps=150, batch_size=16, n_features=None):
        self.X = X
        self.y = y
        self.n_steps = n_steps
#         self.step_size? = step_size
        self.current_pos = 0
        self.n_features = X.shape[1]
        self.batch_size = batch_size
        self.iter = 0
    
    def reset_index(self):
        self.current_pos = 0

    # generates batch of training data
    def generator(self):
        while True: 
            samples = np.zeros((self.batch_size, self.n_steps, self.n_features))
            targets = np.zeros(self.batch_size,)
            
            indices = random.sample(range(150, self.X.shape[0]), self.batch_size)
#             print(self.X.shape, self.y.shape)
            for j, ind in enumerate(indices):
                
                samples[j] = self.X[ind - 150:ind]
#                 try:
                targets[j] = self.y[ind - 1]
#                 except:
#                     targets[j] = self.data[len(self.data) - 1, 1]
#             samples = self.scale_batch(samples)
            yield samples, targets


    # returns standard features for segment
    def extract_features(self, z):        
        x = np.c_[z.mean(axis=1), 
                    z.min(axis=1),
                    z.max(axis=1),
                    z.std(axis=1)]
        return x
    
    # normalizes batch of training features
    def scale_batch(self, samples):
        samples = samples.reshape(-1,self.n_features)
        samples = preprocessing.scale(samples)
        return samples.reshape(self.batch_size, self.n_steps, self.n_features)


    # creates an X-training feature matrix on which prediction must be made
    def create_X(self, ind=150):
#         try:
#             x = self.X[ind-(self.step_size*self.n_steps):ind:,0].reshape(self.n_steps,-1)
        x = self.X[ind-150:ind]
#         except:
#             i = (self.data.shape[0]-self.current_pos) // self.n_steps * self.n_steps
#             x = self.data[self.current_pos:self.current_pos+i:,0].reshape(self.n_steps,-1)
        return x
#         return np.c_[self.extract_features(x),
#                      self.extract_features(x[:,-(self.step_size // 10):]),
#                      self.extract_features(x[:,-(self.step_size // 100):])]

    #   fourier transform does not perform well at the moment...
    #                      np.apply_along_axis(features.create_FFT_feature, 1, x, 4000000)[:,0]]


In [15]:
# split training data into training and validationset
train_X, valid_X = validation_split(scaled_X, 0.8)
train_y, valid_y = validation_split(y, 0.8)

train_gen = KerasDataGenerator(train_X, train_y, n_steps=150 , batch_size=64)
valid_gen = KerasDataGenerator(valid_X, valid_y, n_steps=150,  batch_size=64)
# train_gen.next()

In [94]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Flatten, CuDNNGRU, GRU
from keras.layers.normalization import BatchNormalization
from keras.layers import Dropout
from keras.optimizers import adam, RMSprop
from keras.callbacks import ModelCheckpoint


In [95]:
# steps_per_epoch = len(train_set)//(train_gen.batch_size* train_gen.step_size * train_gen.n_steps)
# print(steps_per_epoch)
# validation_steps = len(valid_set)//(valid_gen.batch_size * valid_gen.step_size * valid_gen.n_steps)
# validation_steps


In [96]:
def create_model(i):
    modelname = "model0" + str(i) + ".hdf5"
    cb = [ModelCheckpoint(modelname, save_best_only=True, period=3)]

    model = Sequential()
    model.add(LSTM(48, input_shape=(None, 6)))
    model.add(Dropout(0.2))
    model.add(Dense(10, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(1))

    model.summary()

    # Compile and fit model
    model.compile(optimizer=adam(lr=0.0005), loss="mae")
    return model, cb


In [97]:
fold_indices = {}
i=0
for train_index, test_index in kf.split(scaled_X):
    i+=1
    #     print("TRAIN:", train_index, "TEST:", test_index)
    train_X, valid_X = scaled_X[train_index], scaled_X[test_index]
    train_y, valid_y = y[train_index], y[test_index]
    
    fold_indices[str(i)] = (train_index, test_index)
    
#     train_gen = KerasDataGenerator(train_X, train_y, n_steps=150 , batch_size=64)
#     valid_gen = KerasDataGenerator(valid_X, valid_y, n_steps=150,  batch_size=64)
    
    
#     model, cb = create_model(i)
#     history = model.fit_generator(train_gen.generator(),
#                               steps_per_epoch=1000,
#                               epochs=30,
#                               verbose=2,
#                               callbacks=cb,
#                               validation_data=valid_gen.generator(),
#                               validation_steps=200)
#     plt.plot(history.history['loss'])
#     plt.plot(history.history['val_loss'])
#     plt.title('Model loss')
#     plt.ylabel('Loss')
#     plt.xlabel('Epoch')
#     plt.legend(['Train', 'Test'], loc='upper left')
#     plt.show()

In [98]:
# model.save('models/fourth_model.h5')
# Plot training & validation loss values


In [99]:
print(fold_indices)

{'1': (array([125829, 125830, 125831, ..., 629142, 629143, 629144]), array([     0,      1,      2, ..., 125826, 125827, 125828])), '2': (array([     0,      1,      2, ..., 629142, 629143, 629144]), array([125829, 125830, 125831, ..., 251655, 251656, 251657])), '3': (array([     0,      1,      2, ..., 629142, 629143, 629144]), array([251658, 251659, 251660, ..., 377484, 377485, 377486])), '4': (array([     0,      1,      2, ..., 629142, 629143, 629144]), array([377487, 377488, 377489, ..., 503313, 503314, 503315])), '5': (array([     0,      1,      2, ..., 503313, 503314, 503315]), array([503316, 503317, 503318, ..., 629142, 629143, 629144]))}


In [100]:
train_index, test_index = fold_indices['4']

In [101]:
train_X, valid_X = scaled_X[train_index], scaled_X[test_index]
train_y, valid_y = y[train_index], y[test_index]
    

In [102]:
train_gen = KerasDataGenerator(train_X, train_y, n_steps=150 , batch_size=64)
valid_gen = KerasDataGenerator(valid_X, valid_y, n_steps=150,  batch_size=64)
# plt.plot(train_y)

In [ ]:
model, cb = create_model(7)
history = model.fit_generator(train_gen.generator(),
                          steps_per_epoch=1000,
                          epochs=50,
                          verbose=2,
                          callbacks=cb,
                          validation_data=valid_gen.generator(),
                          validation_steps=200)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 48)                10560     
_________________________________________________________________
dropout_17 (Dropout)         (None, 48)                0         
_________________________________________________________________
dense_13 (Dense)             (None, 10)                490       
_________________________________________________________________
dropout_18 (Dropout)         (None, 10)                0         
_________________________________________________________________
dense_14 (Dense)             (None, 1)                 11        
Total params: 11,061
Trainable params: 11,061
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
 - 341s - loss: 3.0270 - val_loss: 1.6199
Epoch 2/50
 - 371s - loss: 2.5718 - val_loss: 1.3935
Epoch 3/50
 - 424s - loss: 2.528

In [ ]:
from os import listdir
from os.path import isfile, join

In [ ]:
def create_submission(model, test_path):
    segments = [f for f in listdir(test_path) if isfile(join(test_path, f))]
    predictions = pd.DataFrame(columns=['seg_id','time_to_failure'])
    
    X = []
    for seg_id in segments[:10]:
        file = 'test/' + seg_id
        testdata = pd.read_csv(file).values
        
#         print(features.create_FFT_feauture(testdata[0])
        testobj = KerasDataGenerator(testdata)
        
        test_X = testobj.create_X()
        X.append(test_X)
    return X




#         print(X)
        
        
#         df = pd.DataFrame([[seg_id[:-4], time]], columns=['seg_id','time_to_failure'])
#         predictions = predictions.append(df, ignore_index=True)
        
#     time = model.predict(np.expand_dims(test_X,0))[0][0]

#     predictions.to_csv("submissions/submission04.csv", index=False)

In [ ]:
# model = load_model('models/third_model.h5')

In [ ]:
# model.predict
X = create_submission(model, 'test')

In [ ]:
test_segs = np.c_[X]
test_segs[0]